In [ ]:
import os
os.environ["WANDB_DISABLED"]="true"

In [ ]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] -q

     |████████████████████████████████| 431 kB 27.1 MB/s 
     |████████████████████████████████| 4.9 MB 58.7 MB/s 
     |████████████████████████████████| 116 kB 76.0 MB/s 
     |████████████████████████████████| 1.3 MB 54.3 MB/s 
     |████████████████████████████████| 163 kB 75.9 MB/s 
     |████████████████████████████████| 212 kB 75.1 MB/s 
     |████████████████████████████████| 115 kB 74.6 MB/s 
     |████████████████████████████████| 127 kB 74.8 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 


In [ ]:
# step 1. download data

In [ ]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("wmt16", "tr-en")
metric = load_metric("sacrebleu")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 205756
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [ ]:
import numpy as np
import pandas as pd
import datasets

In [ ]:
train_df=pd.DataFrame([[i['translation']['en'],i['translation']['tr']] for i in raw_datasets['train']],columns=['src','dst'])
val_df=pd.DataFrame([[i['translation']['en'],i['translation']['tr']] for i in raw_datasets['validation']],columns=['src','dst'])
test_df=pd.DataFrame([[i['translation']['en'],i['translation']['tr']] for i in raw_datasets['test']],columns=['src','dst'])
print(len(train_df),len(val_df),len(test_df))

205756 1001 3000


In [ ]:
#create a dataset with scr sentence and target sentence
train_df.head()

,src,dst
0,Kosovo's privatisation process is under scrutiny,Kosova'nın özelleştirme süreci büyüteç altında
1,Kosovo is taking a hard look at its privatisat...,"Kosova, tekrar eden şikayetler ışığında özelle..."
2,By Muhamet Brajshori for Southeast European Ti...,Southeast European Times için Priştine'den Muh...
3,"Feronikel was privatised five years ago, and i...",Feronikel beş yıl önce özelleştirilmesine ve h...
4,"On paper at least, it looks like a great idea.","En azından kağıt üzerinde, harika bir fikir gi..."


In [ ]:
# from the dataframe let create a huggingface data
test_data=datasets.Dataset.from_pandas(test_df)
train_data=datasets.Dataset.from_pandas(train_df)
val_data=datasets.Dataset.from_pandas(val_df)

In [ ]:
#combine all these data into main dataset dictionary
data = datasets.DatasetDict({
    'train': train_data,
    'val':val_data,
    'test':test_data})
data

DatasetDict({
    train: Dataset({
        features: ['src', 'dst'],
        num_rows: 205756
    })
    val: Dataset({
        features: ['src', 'dst'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['src', 'dst'],
        num_rows: 3000
    })
})

In [ ]:
#load tokenizer and model
model_name = "penpen/novel-zh-en"
# from transformers import MarianTokenizer, MarianMTModel
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)

from transformers import AutoTokenizer,AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

loading file source.spm from cache at /root/.cache/huggingface/hub/models--penpen--novel-zh-en/snapshots/ddcbb4bbec89c1446c7b97b4e3be30aee05ad756/source.spm
loading file target.spm from cache at /root/.cache/huggingface/hub/models--penpen--novel-zh-en/snapshots/ddcbb4bbec89c1446c7b97b4e3be30aee05ad756/target.spm
loading file vocab.json from cache at /root/.cache/huggingface/hub/models--penpen--novel-zh-en/snapshots/ddcbb4bbec89c1446c7b97b4e3be30aee05ad756/vocab.json
loading file target_vocab.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--penpen--novel-zh-en/snapshots/ddcbb4bbec89c1446c7b97b4e3be30aee05ad756/tokenizer_config.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--penpen--novel-zh-en/snapshots/ddcbb4bbec89c1446c7b97b4e3be30aee05ad756/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--penpen--novel-zh-en/snapshots/ddcbb4bbec8

In [ ]:

prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "tr"

#write a preprocess function, which take input and targer and tokenize it
def preprocess_function(examples):
    inputs =examples['src']
    targets = examples['dst']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]#put labels in the variable having input to make a single variable
    return model_inputs

In [ ]:
preprocess_function(data['test'][:2])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[66, 727, 254, 337, 5689, 2, 7, 10906, 589, 9868, 672, 78, 104, 11504, 22, 8, 7, 21674, 11923, 4867, 1240, 3253, 1490, 589, 3804, 7828, 0], [66, 12, 727, 254, 1119, 2, 174, 7, 8938, 10532, 18, 480, 95, 1490, 12097, 480, 7, 6436, 5613, 466, 2, 2482, 23102, 177, 6400, 7, 8528, 589, 9868, 672, 10033, 13401, 800, 5283, 26513, 9224, 6331, 804, 5128, 10532, 169, 7, 18043, 7, 6436, 8193, 174, 15754, 1232, 1490, 589, 3804, 7828, 6, 7, 22043, 1232, 22, 204, 250, 8, 400, 7, 10611, 8, 3, 3253, 7, 5561, 4694, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[44144, 1199, 8543, 1232, 50444, 21, 373, 11179, 17123, 43940, 1874, 57546, 3908, 50544, 12950, 1, 746, 57546, 7, 54624, 12643, 12626, 5170, 1

In [ ]:
tokenized_datasets = data.map(preprocess_function, batched=True,remove_columns=data.column_names["train"])

  0%|          | 0/206 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 205756
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
batch_size=12
args = Seq2SeqTrainingArguments(
   output_dir="./mymodel",
  group_by_length=True,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  gradient_accumulation_steps=4,
  evaluation_strategy="epoch",
  num_train_epochs=2,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-4,
  save_total_limit=2,
  predict_with_generate=True    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]#ValueError: Got a string but expected a list instead: 'Norveç'in rakfisk'i: Dünyanın en kokulu bal bu mu?'
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
   
    result = {"bleu": result["score"]}
   
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 205756
  Num Epochs = 2
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 4
  Total optimization steps = 8572


Epoch,Training Loss,Validation Loss,Bleu
0,0.995600,1.720004,6.921028
1,0.744900,1.493166,9.507468


Saving model checkpoint to ./mymodel/checkpoint-500
Configuration saved in ./mymodel/checkpoint-500/config.json
Model weights saved in ./mymodel/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./mymodel/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./mymodel/checkpoint-500/special_tokens_map.json
added tokens file saved in ./mymodel/checkpoint-500/added_tokens.json
Saving model checkpoint to ./mymodel/checkpoint-1000
Configuration saved in ./mymodel/checkpoint-1000/config.json
Model weights saved in ./mymodel/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./mymodel/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./mymodel/checkpoint-1000/special_tokens_map.json
added tokens file saved in ./mymodel/checkpoint-1000/added_tokens.json
Saving model checkpoint to ./mymodel/checkpoint-1500
Configuration saved in ./mymodel/checkpoint-1500/config.json
Model weights saved in ./mymodel/checkpoint-1500/pytorch_model.bin
token

TrainOutput(global_step=8572, training_loss=1.1197710655806947, metrics={'train_runtime': 3747.716, 'train_samples_per_second': 109.803, 'train_steps_per_second': 2.287, 'total_flos': 6594485158674432.0, 'train_loss': 1.1197710655806947, 'epoch': 2.0})

In [ ]:
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")


Saving model checkpoint to ./saved_model
Configuration saved in ./saved_model/config.json
Model weights saved in ./saved_model/pytorch_model.bin
tokenizer config file saved in ./saved_model/tokenizer_config.json
Special tokens file saved in ./saved_model/special_tokens_map.json
added tokens file saved in ./saved_model/added_tokens.json
tokenizer config file saved in ./saved_model/tokenizer_config.json
Special tokens file saved in ./saved_model/special_tokens_map.json
added tokens file saved in ./saved_model/added_tokens.json


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/source.spm',
 './saved_model/target.spm',
 './saved_model/added_tokens.json')

In [ ]:
from transformers import MarianTokenizer, MarianMTModel
tokenizer = MarianTokenizer.from_pretrained('./saved_model')
model = MarianMTModel.from_pretrained('./saved_model')


def translate(text):
    batch = tokenizer([text], return_tensors="pt"  )
    generated_ids = model.generate(**batch)
    result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return result

loading file source.spm
loading file target.spm
loading file vocab.json
loading file target_vocab.json
loading file tokenizer_config.json
loading file added_tokens.json
loading file special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Adding <TERM> to the vocabulary
Adding <GLOS> to the vocabulary
Adding </GLOS> to the vocabulary
Adding MASK to the vocabulary
loading configuration file ./saved_model/config.json
Model config MarianConfig {
  "_name_or_path": "penpen/novel-zh-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      65000
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "